In [2]:
import tensorflow as tf

In [54]:
tf.random.set_seed(5)

In [3]:
x = tf.ones((2,2))

In [4]:
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)

In [5]:
dz_dy = t.gradient(z,y)

In [6]:
dz_dy

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [7]:
dz_dx = t.gradient(z,x)

RuntimeError: A non-persistent GradientTape can only be used tocompute one set of gradients (or jacobians)

In [8]:
dz_dx

NameError: name 'dz_dx' is not defined

In [7]:
tf.reduce_sum(x)

<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [68]:
forward_x = tf.constant([[[1,2,3],
                         [1,1,1]],
                        [[1,1,1],
                        [1,1,1]]], dtype = tf.float32)
with tf.GradientTape() as tt:
    tt.watch(forward_x)
    a = tf.reduce_sum(forward_x, axis = [2,1])
    b = tf.multiply(a,a)

In [9]:
a

NameError: name 'a' is not defined

In [70]:
b

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([81., 36.], dtype=float32)>

In [66]:
db_da = tt.gradient(b, a)

In [67]:
db_da

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([18., 12.], dtype=float32)>

In [71]:
db_dx = tt.gradient(b,forward_x)

In [72]:
db_dx

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[18., 18., 18.],
        [18., 18., 18.]],

       [[12., 12., 12.],
        [12., 12., 12.]]], dtype=float32)>

In [42]:
[1] * 4

[1, 1, 1, 1]

### Broadcast

In [73]:
g = tf.constant([18,12], dtype=tf.float32)

In [74]:
g

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([18., 12.], dtype=float32)>

In [75]:
# tf.broadcast_to(g,forward_x.shape)
g_p = tf.reshape(g,[2,1,1])

In [76]:
g_p

<tf.Tensor: shape=(2, 1, 1), dtype=float32, numpy=
array([[[18.]],

       [[12.]]], dtype=float32)>

In [77]:
tf.tile(g_p, [1,2,3])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[18., 18., 18.],
        [18., 18., 18.]],

       [[12., 12., 12.],
        [12., 12., 12.]]], dtype=float32)>

In [24]:
reduce_test = tf.constant([[[1,3,1],[2,3,1]],
                            [[1,1,1], [2,2,2]]],dtype=tf.float32)

In [25]:
reduce_test.shape

TensorShape([2, 2, 3])

In [26]:
tf.reduce_sum(reduce_test,-1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5., 6.],
       [3., 6.]], dtype=float32)>

In [35]:
tf.reshape(reduce_test,-1)

<tf.Tensor: shape=(12,), dtype=float32, numpy=array([1., 3., 1., 2., 3., 1., 1., 1., 1., 2., 2., 2.], dtype=float32)>

In [36]:
xx = tf.constant([1,2,3], dtype=tf.float32)

In [38]:
tf.broadcast_to(xx, [3,3])

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]], dtype=float32)>

In [41]:
tile_x = tf.reshape(xx,[1,3])

In [42]:
tf.tile(tile_x, [3,1])

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]], dtype=float32)>

### SliceGradient

In [80]:
slice_x = t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]],dtype=tf.float32)

In [93]:
with tf.GradientTape() as tt:
    tt.watch(slice_x)
    a = tf.slice(slice_x, [1, 0, 0], [1, 1, 3])
    b = tf.multiply(a,a/2)

In [95]:
db_da = tt.gradient(b,a)

In [96]:
db_da

<tf.Tensor: shape=(1, 1, 3), dtype=float32, numpy=array([[[3., 3., 3.]]], dtype=float32)>

In [91]:
da_dx = tt.gradient(a,slice_x)

In [92]:
da_dx

<tf.Tensor: shape=(3, 2, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)>

In [86]:
a

<tf.Tensor: shape=(1, 1, 3), dtype=float32, numpy=array([[[3., 3., 3.]]], dtype=float32)>

### ReluGradient

In [32]:
relu_x = tf.constant([-2,0,3],dtype=tf.float32)

In [33]:
with tf.GradientTape() as tape:
    tape.watch(relu_x)
    a = tf.nn.relu(relu_x)

In [34]:
a

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 3.], dtype=float32)>

In [35]:
da_dx = tape.gradient(a,relu_x)

In [36]:
print(da_dx)

tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)


In [ ]:
relu_m = tf.random.normal([])

### ConcatGradient

In [45]:
t1 = tf.constant([[1, 2, 3], [4, 5, 6]],dtype=tf.float32)
t2 = tf.constant([[7, 8, 9], [10, 11, 12]],dtype=tf.float32)


In [46]:
with tf.GradientTape() as concat:
    concat.watch(t2)
    r = tf.concat([t1,t2], 0)

In [41]:
dr_dt1 = concat.gradient(r,t1)

In [42]:
dr_dt1

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [47]:
dr_dt2 = concat.gradient(r,t2)

In [48]:
dr_dt2

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

### SigmoidCrossEntrophy

In [63]:
logits = tf.constant([1., -1., 0., 1., -1., 0., 0.],dtype=tf.float32)
labels = tf.constant([0, 0, 0, 1, 1, 1, 0.5],dtype=tf.float32)

In [64]:
inpus = tf.constant([1,2,3,4,5,6,7],dtype=tf.float32)

In [65]:
with tf.GradientTape() as sigmoid_cross:
    sigmoid_cross.watch(logits)
    s_c = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels,logits=logits)

In [66]:
sigmoid_cross.gradient(s_c,logits)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([ 0.7310586 ,  0.26894143,  0.5       , -0.26894143, -0.7310586 ,
       -0.5       ,  0.        ], dtype=float32)>

In [53]:
s_c

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([1.3132617, 0.3132617, 0.6931472, 0.3132617, 1.3132617, 0.6931472,
       0.6931472], dtype=float32)>